In [41]:
import plotly.express as px

In [42]:
class Agent:
    def __init__(self, start_position, end_position):
        self.start = start_position
        self.end = end_position
        self.current_node = None
        self.node_position = None


In [43]:
class Node():
    
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def __repr__(self) -> str:
        return self.position


In [44]:
def astar(maze, agent1, agent2):
    
    new_maze = maze
    
    # Create agent1 start and end node
    start_node1 = Node(None, agent1.start)
    start_node1.g = start_node1.h = start_node1.f = 0
    end_node1 = Node(None, agent1.end)
    end_node1.g = end_node1.h = end_node1.f = 0
    
    # Create agent2 start and end node
    start_node2 = Node(None, agent2.start)
    start_node2.g = start_node2.h = start_node2.f = 0
    end_node2 = Node(None, agent2.end)
    end_node2.g = end_node2.h = end_node2.f = 0

    # Initialize both open and closed list
    open_list1  = []
    closed_list1 = []
    open_list2  = []
    closed_list2 = []

    # Add the agent1.start node
    open_list1.append(start_node1)
    open_list2.append(start_node2)

    # Loop until you find the end
    while len(open_list1) + len(open_list2) > 0:

        # Get the agent 1 current node
        agent1.current_node = open_list1[0]
        for item in open_list1:
            # print("openList:" ,item.position[0],  item.position[1] )
            new_maze[item.position[0]][item.position[1]] = 6 # change openlist items colors
            if item.f < agent1.current_node.f:
                agent1.current_node = item
        # print("we are here: ", agent1.current_node.position, agent1.current_node.g)
        
        # Get the agent 2 current node
        agent2.current_node = open_list2[0]
        for item in open_list2:
            # print("openList:" ,item.position[0],  item.position[1] )
            new_maze[item.position[0]][item.position[1]] = 6 # change openlist items colors
            if item.f < agent2.current_node.f:
                agent2.current_node = item
        # print("we are here: ", agent2.current_node.position, agent2.current_node.g)


        
        
        # Pop current off open list, add to closed list
        open_list1.remove(agent1.current_node)
        closed_list1.append(agent1.current_node)
        
        # Pop current off open list, add to closed list
        open_list2.remove(agent2.current_node)
        closed_list2.append(agent2.current_node)



        # Found the goal
        if agent1.current_node == end_node1:
            path = []
            current = agent1.current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            final_path = path[::-1] 
            
            print(final_path) # Print reversed path
            for path_position in final_path:
                new_maze[path_position[0]][path_position[1]] = 8
            return(new_maze)
        
        
        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0),(-1, -1), (1, 1), (-1, 1), (1, -1)]: # Adjacent squares

            # Get node position
            agent1.node_position = (agent1.current_node.position[0] + new_position[0], agent1.current_node.position[1] + new_position[1])

            # Make sure within range
            if agent1.node_position[0] > (len(maze) - 1) or agent1.node_position[0] < 0 or agent1.node_position[1] > (len(maze[len(maze)-1]) -1) or agent1.node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[agent1.node_position[0]][agent1.node_position[1]] != 5:
                continue

            # Create new node
            new_node = Node(agent1.current_node, agent1.node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            if child in closed_list1:
                continue

            # Create the f, g, and h values
            child.g = agent1.current_node.g + 1
            child.h = ((child.position[0] - end_node1.position[0]) ** 2) + ((child.position[1] - end_node1.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            if child in open_list1:
                i = open_list1.index(child)
                duplicated = open_list1[i]
                if child.g > duplicated.g:
                    continue


            # Add the child to the open list
            open_list1.append(child)



In [45]:
def main():

    maze = [[5, 0, 5, 0, 5, 5, 5, 5, 5, 5],
            [5, 0, 5, 0, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
            [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]


        
    
    start1 = (0, 0)
    end1 = (0, 8)
    agent1 = Agent(start1, end1)
    
    start2 = (9, 9)
    end2 = (9, 1)
    agent2 = Agent(start1, end1)

    new_maze = astar(maze, agent1, agent2)
    
    

    fig = px.imshow(new_maze, color_continuous_midpoint=0.0, range_color= [0,10], color_continuous_scale = 'IceFire' )
    fig.show()


In [46]:
main()

IndexError: list index out of range